<a href="https://colab.research.google.com/github/jstarace/CS_504_Assignment3_Vakanski/blob/main/CS504_Vakanski_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

### The following is a complete walk-through of Assignment #3 for CS 504: ST Adversarial Machine Learning, taught by Dr. Vakanski at the University of Idaho in the Spring of 2023.  The assignment is being made available to assist in the understanding of future related assignments at the request of Dr. Vakanski.

### The assignment is broken down into 2 parts.
1. Adversarial Traning defense against evasion attacks
1. Poisoning attacks against deep learning classification models

For this assignment I used Google Colab for the training of the models and completion of the assignment.  It is recommend that the same or similar is used.

# Notebook Setup

## Setup and Configuration

Before diving into the assignment we want to make sure our environment has all the necessary libraries and connections

In [4]:
# Connect to Google Drive.
# All images required for the assignment were either provided by Dr. Vakanski or were publicly available
# Images were loaded onto google drive for ease of access

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Install the adversarial robustness toolbox so that we can work with adversarial models
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 34.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


### Imports

In [6]:
# Import the following libraries

import os
import cv2
import sys
import keras
import random
import imageio
import tarfile
import natsort
import warnings
import numpy as np
import tensorflow as tf
import keras.backend as k
import matplotlib.pyplot as plt

In [7]:
# From installed and available libraries import these specific classes

from os import listdir

from keras.models import Model
from keras.preprocessing import image
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping 
from keras.applications.imagenet_utils import preprocess_input

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.tools.docs.doc_controls import T
from tensorflow.keras.utils import to_categorical
from tensorflow.python.ops.math_ops import TruncateDiv
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, GlobalAveragePooling2D

from art.estimators.classification import KerasClassifier
from art.estimators.classification import SklearnClassifier

from art.attacks.evasion import BoundaryAttack
from art.attacks.evasion import FastGradientMethod,ProjectedGradientDescent,DeepFool
from art.utils import to_categorical

from skimage.util import compare_images
from IPython.display import clear_output
from matplotlib.gridspec import GridSpec
from skimage import data, transform, exposure

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [8]:
# This line is important, without it the attacks in the ART toolbox won't work
# And it needs to be run before training the models
tf.compat.v1.disable_eager_execution()

# Supress warnings
warnings.filterwarnings('ignore')

In [9]:
# Print the version of tensorflow and keras
print("TensorFlow version:{}".format(tf.__version__))
print("Keras version:{}".format(keras.__version__))

TensorFlow version:2.12.0
Keras version:2.12.0


# Part 1

## Section 1

## Section 2

## Section 3

## Section 4

## Section 5

# Part 2